# StarGAN  - New Dataset ( Youtube and Wiki )

## Wiki Dataset

Downloading the necessary package and create folder for storing the wiki data.

In [1]:
!pip install wikipedia


!mkdir /content/musician
%cd /content/musician

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=29fec25a1ecc655bb118ebe32e6de0214ebf531cbfb756d3b65bb84b04bca05f
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
/content/musician


Get images from wiki url and download images.

**Note -This will run for quite long, you can manually stop it anytime you need.**

In [0]:
import wikipedia
import urllib.request
import numpy as np

music = wikipedia.page("Lists_of_musicians").links

ext = (".jpg",".png")
print(len(music))
for music_specific in music:
    artist =wikipedia.page(music_specific).links
    # np.random.suffle(artist)
    for page in artist[:round(len(artist)/10)]:
      try:
        wikipage = wikipedia.page(page)
        print("Page Title: %s" % wikipage.title)
        print ("Page URL: %s" % wikipage.url)
        print (" - Main Image: %s" % wikipage.images[0])
        for count,image in enumerate(wikipage.images):
          if image.endswith(ext):
            urllib.request.urlretrieve(image,page+str(count)+".jpg")
        print ("")
      except:
        pass


        

449
Page Title: 1976
Page URL: https://en.wikipedia.org/wiki/1976
 - Main Image: https://upload.wikimedia.org/wikipedia/commons/9/9c/20111015_174748_phil-ochs_%28cropped%29.jpg

Page Title: 1982
Page URL: https://en.wikipedia.org/wiki/1982
 - Main Image: https://upload.wikimedia.org/wikipedia/commons/f/fd/110818_Gemma_Chan_in_an_interview_for_Collider_Video.png

Page Title: 2012 Summer Olympics closing ceremony
Page URL: https://en.wikipedia.org/wiki/2012_Summer_Olympics_closing_ceremony
 - Main Image: https://upload.wikimedia.org/wikipedia/commons/7/75/2012_Summer_Olympics_closing_ceremony.jpg

Page Title: 2gether (band)
Page URL: https://en.wikipedia.org/wiki/2gether_(band)
 - Main Image: https://upload.wikimedia.org/wikipedia/en/d/d6/2ge%2Bher.jpg

Page Title: 3T
Page URL: https://en.wikipedia.org/wiki/3T
 - Main Image: https://upload.wikimedia.org/wikipedia/commons/5/51/3T_in_performance_%28Hannover%2C_1996%29.jpg

Page Title: '50s progression
Page URL: https://en.wikipedia.org/wik

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page Title: Alessandro Besozzi
Page URL: https://en.wikipedia.org/wiki/Alessandro_Besozzi
 - Main Image: https://upload.wikimedia.org/wikipedia/en/8/8a/OOjs_UI_icon_edit-ltr-progressive.svg

Page Title: Alessandro Grandi
Page URL: https://en.wikipedia.org/wiki/Alessandro_Grandi
 - Main Image: https://upload.wikimedia.org/wikipedia/en/8/8a/OOjs_UI_icon_edit-ltr-progressive.svg

Page Title: Alessandro Longo
Page URL: https://en.wikipedia.org/wiki/Alessandro_Longo
 - Main Image: https://upload.wikimedia.org/wikipedia/commons/2/20/Noteicon3.svg

Page Title: Alessandro Marcello
Page URL: https://en.wikipedia.org/wiki/Alessandro_Marcello
 - Main Image: https://upload.wikimedia.org/wikipedia/commons/4/48/Alessandro_Marcello.jpg

Page Title: Alessandro Melani
Page URL: https://en.wikipedia.org/wiki/Alessandro_Melani
 - Main Image: https://upload.wikimedia.org/wikipedia/en/8/8a/OOjs_UI_icon_edit-ltr-progressive.svg

Page Title: Alessandro Nini
Page URL: https://en.wikipedia.org/wiki/Alessandro_

## Video Dataset

### Installing necessary package 


inspired by the code https://github.com/ahmetozlu/face_recognition_crop

In [1]:
!pip install pytube
  

### Importing the necessary package

In [2]:
!rm -r video
!mkdir video

rm: cannot remove 'video': No such file or directory


In [0]:
import urllib
from bs4 import BeautifulSoup
from pytube import YouTube
import cv2
import os
import glob

### getting url from youtube video search list

In [0]:
def get_urls(text, limit=10):
  query = urllib.parse.quote(text)
  url = "https://www.youtube.com/results?search_query=" + query
  response = urllib.request.urlopen(url)
  html = response.read()
  soup = BeautifulSoup(html, 'html.parser')
  urls = []
  for i, vid in enumerate(soup.findAll(attrs={'class':'yt-uix-tile-link'})):
      if i < limit:
          urls.append('https://www.youtube.com' + vid['href'])
  print(f"Found {len(urls)} video links for {text}")
  return urls

### Download Video

In [0]:

def download_video(url, path=None, max_duration=3600):
  try:
    yt = YouTube(url)
    duration = int(yt.player_config_args['player_response']['streamingData']['formats'][0]['approxDurationMs'])
    if duration < max_duration*60*1000:
        yt = yt.streams.filter(file_extension='mp4').first()
        out_file = yt.download(path)
        file_name = out_file.split("\\")[-1]
        print(f"Downloaded {file_name} correctly!")
    else:
        print(f"Video {url} too long")
  except Exception as exc:
      print(f"Download of {url} did not work because of {exc}...")

###Labeling the folder 

In [0]:
def max_label(name, folder):
    '''Look at a folder and check the files with pattern "name_###.jpg" to extract the
    largest label present.'''
    
    path_pattern = os.path.join(folder, name + "_*.jpg")
    existing_files = glob.glob(path_pattern)
    if not existing_files:
        biggest_label = 0
    else:
        existing_labels = map(lambda s: int(s.split('_')[-1].split('.')[0]), existing_files)
        biggest_label = max(existing_labels)
    return biggest_label

### Extract Image 

In [0]:
def extract_images_from_video(video, folder=None, delay=30, name="video", max_images=200, silent=False):    
    vidcap = cv2.VideoCapture(video)
    count = 0
    num_images = 0
    if not folder:
        folder = os.getcwd()
    label = max_label(name, folder)
    success = True
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    
    while success and num_images < max_images:
        success, image = vidcap.read()
        num_images += 1
        label += 1
        file_name = name + "_" + str(label) + ".jpg"
        path = os.path.join(folder, file_name)
        cv2.imwrite(path, image)
        if cv2.imread(path) is None:
            os.remove(path)
        else:
            if not silent:
                print(f'Image successfully written at {path}')
        count += delay*fps
        vidcap.set(1, count)

In [0]:

def extract_images_from_word(text, delete_video=False, image_delay=30, 
                             num_urls=15, max_images=300, name="video", max_duration=60, silent=False):
    if not os.path.exists(name):
        os.mkdir(name)
    urls = get_urls(text, num_urls)
    for url in urls:
        download_video(url, max_duration=max_duration)
    for i, video in enumerate(glob.glob("*.mp4")):
        extract_images_from_video(video, folder=name, delay=image_delay, name=name, max_images=max_images, silent=silent)
        if delete_video:
            os.remove(video)

### Implementing video extractor

In [9]:
extract_images_from_word('Freinds and Big bang theory')
extract_images_from_word('money heist')

Found 15 video links for Freinds and Big bang theory
Download of https://www.youtube.com/channel/UCQGNGhpDq-HGBKaP4jfez5Q did not work because of This video is unavailable....
Download of https://www.youtube.com/channel/UC3-DfztgFS4vMwjQPUB7SPg did not work because of This video is unavailable....
Download of https://www.youtube.com/channel/UCwvgBozI-m1QJ5BVowF_xHg did not work because of This video is unavailable....
Downloaded /content/10 Scenes From Big Bang Theory Which Are Copied From FRIENDS.mp4 correctly!
Downloaded /content/The Big Bang Theory Cast Interview - Tribute To Director James Burrows.mp4 correctly!
Downloaded /content/Ross Geller Vs Howard Wolowitz Reaction to pregnancy Friends Vs The big bang theory.mp4 correctly!
Downloaded /content/2016 &quotFriends&quot Reunion - EXTENDED!!!.mp4 correctly!
Downloaded /content/Friends And The Big Bang Theory Similarities TOTI.mp4 correctly!
Downloaded /content/The Big Bang theory-Hilarious bloopers (part-1).mp4 correctly!
Downloade

In [10]:
extract_images_from_word('Game of thrones')

Found 15 video links for Game of thrones
Download of https://www.youtube.com/channel/UCow8-ZftRoZelY710tvO45Q did not work because of This video is unavailable....
Downloaded /content/Best of Game of Thrones - Most Badass Scenes Compilation.mp4 correctly!
Download of https://www.youtube.com/watch?v=6w9kf33-SfA did not work because of This video is unavailable....
Downloaded /content/Arya Stark Kills The Night King  Game of Thrones S08E03.mp4 correctly!
Downloaded /content/Questions left Unanswered after the Game of Thrones Finale.mp4 correctly!
Downloaded /content/The Cast Remembers  Game of Thrones Season 8 (HBO).mp4 correctly!
Downloaded /content/Daenerys Attacks King’s Landing  Game of Thrones S08E05.mp4 correctly!
Download of https://www.youtube.com/user/GameofThrones did not work because of This video is unavailable....
Downloaded /content/Game of Thrones Fans are Losing It over the George RR Martin News! (Nerdist News w Amy Vorpahl).mp4 correctly!
Download of https://www.youtube.

### Zipping and downloading the cropped file 

Note : you dont need to run this block if you are not downloading the images extracted. This block was used to check the extracted images on local system.

In [0]:
!zip -r /content/images.zip /content/video

In [0]:
!zip -FF images.zip --out new-images.zip

In [0]:
from google.colab import files
files.download("/content/new-images.zip")

## AUTOCROPPER

crops just the way we need it to but only works for 20 - 30 % of the images

It used to detect faces from the youtube dataset and wiki dataset , then crop only faces to get new dataset as we need for StarGan

In [12]:
!pip install autocrop

     |████████████████████████████████| 163kB 6.8MB/s 
     |████████████████████████████████| 2.1MB 37.9MB/s 
  Created wheel for autocrop: filename=autocrop-0.3.5-cp36-none-any.whl size=155767 sha256=89bffeceec4a2e22db831d32ff8f0efb5e4ef15096a3bc499ae87d5d21143c14
  Stored in directory: /root/.cache/pip/wheels/b5/47/1a/6bc12bc3ae00ae67b07c11c00f5d9edfaf85749504af60b63a
Successfully built autocrop
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 4.3.0
    Uninstalling Pillow-4.3.0:
      Successfully uninstalled Pillow-4.3.0


In [13]:
%cd /content/
!mkdir autocrop
%cd autocrop/
!mkdir pics
!mkdir crop
!mkdir reject

/content
/content/autocrop


### Copying youtube dataset

In [0]:
!mv /content/video/*.jpg /content/autocrop/pics/

### Copying wiki dataset

In [0]:
!mv /content/musician/*.jpg /content/autocrop/pics/

### Cropping all dataset images.

In [16]:
!autocrop -i pics -o crop -w 800 -H 800 -r reject

Processing images in folder: /content/autocrop/pics
No face detected: /content/autocrop/reject/video_1108.jpg
Face detected:    /content/autocrop/crop/video_357.jpg
Face detected:    /content/autocrop/crop/video_1162.jpg
Face detected:    /content/autocrop/crop/197627.jpg
Face detected:    /content/autocrop/crop/video_677.jpg
No face detected: /content/autocrop/reject/video_507.jpg
No face detected: /content/autocrop/reject/video_1048.jpg
No face detected: /content/autocrop/reject/video_705.jpg
Face detected:    /content/autocrop/crop/video_239.jpg
Face detected:    /content/autocrop/crop/19828.jpg
Face detected:    /content/autocrop/crop/video_237.jpg
No face detected: /content/autocrop/reject/Balkans1.jpg
No face detected: /content/autocrop/reject/video_134.jpg
No face detected: /content/autocrop/reject/video_1187.jpg
No face detected: /content/autocrop/reject/video_516.jpg
No face detected: /content/autocrop/reject/video_967.jpg
Face detected:    /content/autocrop/crop/video_188.jpg

**Zipping and Downloading the cropped images**

Note : dont run this block of code if you dont want to download the images.

In [0]:
!zip -r /content/autocrop/cropped.zip /content/autocrop/crop/

from google.colab import files
files.download("/content/autocrop/cropped.zip")

## Implementation of StarGan

In [17]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 1.8MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.2
    Uninstalling scipy-1.3.2:
      Successfully uninstalled scipy-1.3.2


In [33]:
%cd /content
!rm -r stargan-tensorflow


!git config --global user.name "sanjay adhikari , reuben francis , gautam meenakashi , bhavana ramanna"
!git config --global user.email "<dummy_name>@students.mq.edu.au"

!git clone https://gitlab.com/reubenbf/stargan-tensorflow.git
%cd /content/stargan-tensorflow

/content
Cloning into 'stargan-tensorflow'...
remote: Enumerating objects: 204583, done.
remote: Counting objects: 100% (204583/204583), done.
remote: Compressing objects: 100% (204547/204547), done.
remote: Total 204583 (delta 34), reused 204577 (delta 28)
Receiving objects: 100% (204583/204583), 3.12 GiB | 28.49 MiB/s, done.
Resolving deltas: 100% (34/34), done.
Checking out files: 100% (204641/204641), done.
/content/stargan-tensorflow


### STARGAN testing for new dataset.

In [0]:
!rm -r /content/stargan-tensorflow/dataset/celebA/test
!mkdir /content/stargan-tensorflow/dataset/celebA/test
!cp  /content/autocrop/crop/*.jpg /content/stargan-tensorflow/dataset/celebA/test




In [0]:
!cp  /content/autocrop/crop/*.jpg /content/stargan-tensorflow/New\ Dataset\ /dataset

In [30]:
%cd /content/stargan-tensorflow/
!rm -r results/StarGAN_celebA_wgan-gp_6resblock_6dis/images
!python main.py --phase test

!git add * 
!git commit -m "added dataset"
!git push

/content/stargan-tensorflow
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



2019-11-15 12:23:16.659091: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-11-15 12:23:16.659388: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1e49100 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-11-15 12:23:16.659423: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-11-15 12:23:16.684790: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-15 12:23:16.832645: I ten

## Testing and Questionnaire Preparation.

We select random images from the result of generated images from STARGAN to prepare google questionaire survey.

In [32]:
%cd /content/stargan-tensorflow/
import random 
import glob 
import shutil
file_list = glob.glob('results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/*')

len(file_list)
!mkdir /content/random_image

dst = '/content/random_image'
random_image= random.sample(file_list , 10)
for src in random_image : 
  print(src)
  newPath = shutil.copy(src, dst)
  print(newPath)


/content/stargan-tensorflow
mkdir: cannot create directory ‘/content/random_image’: File exists
results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/video_1130.jpg
/content/random_image/video_1130.jpg
results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/video_1161.jpg
/content/random_image/video_1161.jpg
results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/198217.jpg
/content/random_image/198217.jpg
results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/video_560.jpg
/content/random_image/video_560.jpg
results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/198229.jpg
/content/random_image/198229.jpg
results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/video_45.jpg
/content/random_image/video_45.jpg
results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/video_443.jpg
/content/random_image/video_443.jpg
results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/video_753.jpg
/content/random_image/video_753.jpg
results/StarGAN_celebA_wgan-gp_6resblock_6dis/images/fake_9.png
/content/random_image/fake

### Download randomly generated images.

In [0]:
!zip -r /content/random_image.zip /content/random_image

from google.colab import files
files.download("/content/random_image.zip")